In [2]:
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects

Unable to determine R home: [WinError 2] The system cannot find the file specified


In [26]:
dct = {'method': "ssgsea", 'kcdf': "Gaussian", "abs_ranking": False,
        "min_sz": 1, "max_sz": float('inf'), "parallel_sz": 1, "max_diff": True,
        "tau": False, "ssgsea_norm": True, "verbose": True}
arg_list = []
for val in dct.values():
    arg_list.append(val)

In [6]:
arg_list

['ssgsea', 'Gaussian', False, 1, inf, 1, True, None, True, True]

In [11]:
list(robjects.vectors.ListSexpVector(arg_list)[2])

[False]

In [27]:
x = robjects.ListVector(dct)
x[1]

'Gaussian'


In [3]:
"""
GSVA -> arguments
(method=c("gsva", "ssgsea", "zscore", "plage"),
kcdf=c("Gaussian", "Poisson", "none"),
abs.ranking=FALSE,
min.sz=1,
max.sz=Inf,
parallel.sz=1L,
mx.diff=TRUE,
tau=switch(method, gsva=1, ssgsea=0.25, NA),
ssgsea.norm=TRUE,
verbose=TRUE,
BPPARAM=SerialParam(progressbar=verbose))


SingScore -> simpleScore Arguments
(rankData,
upSet,
downSet = NULL,
subSamples = NULL,
centerScore = TRUE,
dispersionFun = mad,
knownDirection = TRUE
)

AUcell ->  AUCell_buildRankings arguments

(featureType = "genes",
plotStats = TRUE,
splitByBlocks = FALSE,
BPPARAM = NULL,
keepZeroesAsNA = FALSE,
verbose = TRUE,
nCores = NULL,
mctype = NULL)

AUcell ->  AUCell_calAUC arguments
(geneSets,
rankings,
nCores = 1,
normAUC = TRUE,
aucMaxRank = ceiling(0.05 * nrow(rankings)),
verbose = TRUE
)
"""


import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
import pandas as pd
import numpy as np

class r_2_python_conversion:

    def __init__(self, csv_file_path, gmt_file_path):
        # self.r_file_path = r_script_file_path
        self.csv_file_path = csv_file_path
        self.gmt_file_path = gmt_file_path

    def aucell_score(self, r_file_path, **args_list):
        # Defining the R script and loading the instance in Python
        r = robjects.r
        r['source'](r_file_path)  # 'C:\\Users\\hp\\Downloads\\AUcell.R'

        # creating the global environment and import the aucell_data_collection function from r script in globalenv.
        aucell_data_collection = robjects.globalenv['aucell_data_collection']
        as_null = robjects.r['as.null']
        dct = {"featureType": "genes",
                "plotStats": True,
                "splitByBlocks": False,
                "BPPARAM": as_null(),
                "keepZeroesAsNA": False,
                "verbose": True,
                "nCores": as_null(),
                "mctype": as_null(),
                "nCores_cal_auc": 1,
                "normAUC": True,
                "aucMaxRank": 0.05,
                "verbose_cal_auc": True}
        
        for key, value in args_list.items():
            dct[key] = value
        print(dct)

        # store the argument of aucell_data_collection function in tuple
        # tuple contain key and value -> key: argument name  and value: file_paths
        args = (('csv_file_path', self.csv_file_path),
                ('gmt_file_path', self.gmt_file_path),
                ('lst', robjects.ListVector(dct)))

        # pass the args variable in aucell_data_collection function to call the r function in python.
        matrix = aucell_data_collection.rcall(args)

        return np.array(matrix)

    def singscore(self, r_file_path, **args_list):
        # Defining the R script and loading the instance in Python
        r = robjects.r
        r['source'](r_file_path)
        singscore_data_collection = robjects.globalenv['singscore_data_collection']
        mad = robjects.r['mad']
        as_null = robjects.r['as.null']
        dct = {"downSet": as_null(),
                "subSamples": as_null(),
                "centerScore": True,
                "dispersionFun": mad,
                "knownDirection": True}
        
        for key, value in args_list.items():
            if key != "dispersionFun":
                dct[key] = value
            else:
                param = robjects.r[value]
                dct[key] = param
                
        print(dct)
        args = (('csv_file_path', self.csv_file_path),
                ('gmt_file_path', self.gmt_file_path),
                ('lst', robjects.ListVector(dct))
               )
        singscore_matrix = singscore_data_collection.rcall(args)

        # write the total_score into matrix
        rpackages.importr('MASS')
        write_matrix = robjects.r('write.matrix')
        write_matrix(singscore_matrix[0], file="total_score_mat1", sep=",")
        # convert the total_score_mat into csv file
        total_score_df = pd.read_csv("total_score_mat1", sep=",")
        total_score_df['X'] = list(singscore_matrix[2])
        total_score_df.to_numpy()

        write_matrix(singscore_matrix[1], file="total_dispersion_mat1", sep=",")
        total_dispersion_df = pd.read_csv("total_dispersion_mat1", sep=",")
        total_dispersion_df['X'] = list(singscore_matrix[2])
        total_dispersion_df.to_numpy()

        return total_score_df, total_dispersion_df

    def gsva_score(self, r_file_path, **args_list):
        r = robjects.r
        r['source'](r_file_path)
        gsva_data_collection = robjects.globalenv['gsva_data_collection']
        base = rpackages.importr('base')
        switch = robjects.r['switch']
        as_null = robjects.r['as.null']
        dct = {"method": "ssgsea", 
               'kcdf': "Gaussian", 
               "abs_ranking": False,
               "min_sz": 1,
               "max_sz": float('inf'),
               "parallel_sz": 100000000,
               "max_diff": True,
               "tau": False,
               "ssgsea_norm": True,
               "verbose": True}
        for key, value in args_list.items():
            dct[key] = value
                
        print(dct)
        args = (('csv_file_path', self.csv_file_path),
                ('gmt_file_path', self.gmt_file_path),
                ('lst', robjects.ListVector(dct)))

        gsva_es_ma1 = gsva_data_collection.rcall(args)
        return np.array(gsva_es_ma1)
    

switch = robjects.r['switch']
r_path = r'C:\Users\hp\Downloads\Telegram Desktop\GSVA.R'
obj = r_2_python_conversion(r"C:/Users/hp/Downloads/matrix_.csv", r"C:\Users\hp\Downloads\c2.cp.v7.5.1.symbols (1).gmt")
output = obj.gsva_score(r_path, method = "gsva")
print(output)

{'method': 'gsva', 'kcdf': 'Gaussian', 'abs_ranking': False, 'min_sz': 1, 'max_sz': inf, 'parallel_sz': 100000000, 'max_diff': True, 'tau': False, 'ssgsea_norm': True, 'verbose': True}
Estimating GSVA scores for 2982 gene sets.
Estimating ECDFs with Gaussian kernels
  |======================================================================| 100%

[[-0.08409868 -0.49169391  0.48858938 ... -0.28524248  0.43278541
   0.15559759]
 [ 0.16718954 -0.45328354  0.56543996 ... -0.28219792  0.52813443
   0.38614212]
 [ 0.41842732 -0.58573557  0.34857668 ... -0.20950346  0.59973591
   0.12959871]
 ...
 [ 0.47766672  0.20891854  0.39626027 ... -0.0536768  -0.47001619
   0.06009579]
 [-0.07813079 -0.47599722 -0.02010886 ... -0.16682713 -0.20657371
  -0.37369049]
 [ 0.1272252  -0.05545697  0.25015093 ...  0.12339612  0.0034401
   0.3238779 ]]


array([0.15274359, 0.09074874, 0.23607266, 0.13269508, 0.14793988,
       0.18474028, 0.1998443 , 0.16720945, 0.1486899 , 0.16351866,
       0.14404946, 0.08069661, 0.14615439, 0.1144539 , 0.14354557,
       0.2292212 , 0.14348754, 0.10689912, 0.14377548, 0.15968237,
       0.15431819, 0.285204  , 0.15200235, 0.14372052, 0.12448476,
       0.11533976, 0.17815316, 0.14297443, 0.13198856, 0.0948491 ,
       0.15246792, 0.0913329 , 0.12846629, 0.22365252, 0.1226284 ,
       0.19098004, 0.14576105, 0.15547064, 0.19841877, 0.11883833,
       0.25643669, 0.1499168 ])

In [7]:
as_double = robjects.r['as.double']
is_double = robjects.r['is.double']
as_double(1)

1.000000


In [5]:
r = robjects.r
r['source']("F:\GenterpretR Work\R script\CTRP dataset.R")  # 'C:\\Users\\hp\\Downloads\\AUcell.R'

# creating the global environment and import the aucell_data_collection function from r script in globalenv.
ctrp_fun = robjects.globalenv['ctrp']


# pass the args variable in aucell_data_collection function to call the r function in python.
matrix = ctrp_fun
print(matrix)

function(){
  data<-readRDS('F:\\GenterpretR Work\\R script\\CTRPv2.rds')
  data@sensitivity
  res <- data@sensitivity$profiles
  return(res)
}



In [ ]:
def aucell_score(self, r_file_path):
        # Defining the R script and loading the instance in Python
        r = robjects.r
        r['source'](r_file_path)  # 'C:\\Users\\hp\\Downloads\\AUcell.R'

        # creating the global environment and import the aucell_data_collection function from r script in globalenv.
        aucell_data_collection = robjects.globalenv['aucell_data_collection']
        as_null = robjects.r['as.null']
        

        # pass the args variable in aucell_data_collection function to call the r function in python.
        matrix = aucell_data_collection

        